In [ ]:
# %matplotlib inline
%matplotlib qt
import hyperspy.api as hs
import numpy as np
import lmfit as lm
import matplotlib.pyplot as plt
import matplotlib
from espm.models import edxs
from espm.models.EDXS_function import print_concentrations_from_W
import espm.spectrum_fitting as sf
from espm.conf import DATASETS_PATH
from pathlib import Path
from espm.datasets import spim
from espm.estimators import SmoothNMF

# Guide to this notebook

Every time the symbol ⚠️ appears, a user input is required in the cell below.

If a cell made a few windows pop up, please kill the windows once you're done. Otherwise every new plot will be displayed in top of the other (which may slow your computer down). 

Overview of the different steps :
- I. Load the data either a spectrum (go to step III.) or a spectrum image
- II. Select an area of interest in your spectrum image.
- III. Creates an energy scale (x) from your data for the fitting procedure.
- IV. Selects the regions of the spectrum were there are no peaks for background fitting.
- V. Creates the required objects (partial x and y) for the background fitting procedure.
- VI. Fitting the background. If this is not satisfactory go back to IV.

## I. Load spectrum & energy scale

The spectrum is expected to be an average spectrum, its energy range is expected to start above 0.0

⚠️

In [ ]:
input_filename = str("phyllo_spectre_moyen.hspy")
spectrum = hs.load(input_filename)

offset = spectrum.axes_manager[0].offset
scale = spectrum.axes_manager[0].scale
size = spectrum.axes_manager[0].size

x = np.linspace(offset,size*scale+offset,num = size)

# II. Select areas for bckgd fitting

⚠️ Input in the ``span_number`` the number of selection areas you wish to use. Once you execute the cell, a window will pop with a few green areas and a red spectrum. The green areas correspond to the regions were the background will be fitted. You can, drag, enlarge and reduce these green areas.

In [ ]:
span_number = 4

def selection_areas(number,spectrum,scale) :
    spectrum.plot()
    size = spectrum.axes_manager[0].size //(2*number)
    roi_list = []
    for i in range(number) :
        roi_list.append(hs.roi.SpanROI(offset+ size*scale + 2*i*size*scale, 2*size*scale + offset+ 2*i*size*scale))
        roi_list[-1].interactive(spectrum)
    return roi_list

spans = selection_areas(span_number,spectrum,scale)

Once you are satisfied with the selected areas, save their positions using the cell below

In [ ]:
list_energies = [[roi.left,roi.right] for roi in spans]

# III. Fitting the continuum

⚠️ Input the sample parameters below and execute the cell to fit

In [ ]:
thickness = 50e-7
density = 5.11
take_off_angle = 24
elements_dict = {"Sr" : 0.25, "Ti" : 0.25, "O" : 0.5}
detector = "SDD_efficiency.txt"

#################################################################################################
# For a custom detector, uncomment below and replace "SDD_efficiency.txt" with the dictionnary. #
#################################################################################################

# {"detection" : {
#     "thickness" : 450e-4,
#     "elements_dict" : {
#         "Si" : 1.0
#     }
# },"layer" : {
#     "thickness" : 10e-7,
#     "elements_dict" : {
#         "Si" : 0.33,
#         "O" : 0.66,
#         "Al" : 1.0
#     }
# }}

part_x, part_y, sum_boola = sf.make_partial_xy(list_energies,spectrum,x)

example = {
    "b0" : 1.0,
    "b1" : 1.0,
    "params_dict" : {
    "Det" : detector,
    'Abs' : {
        "thickness" : thickness,
        "toa" : take_off_angle,
        "density" : density,
        "atomic_fraction" : True},
        "elements_dict" : elements_dict
    }
}

pars = sf.ndict_to_params(example)

#################################################################################################
# You can uncomment the lines below if you want to add constraints to the absorption parameters #
#################################################################################################

# pars["params_dict__Abs__thickness"].vary = False
# pars["params_dict__Abs__thickness"].max = 5000e-7
# pars["params_dict__Det__layer__thickness"].vary = False
pars["params_dict__Abs__toa"].vary = False
pars["params_dict__Abs__density"].vary = False

out = lm.minimize(sf.residual, pars, args=(part_x,), kws={'data': part_y})
print(lm.fit_report(out))

### Plotting the results

The red curves corresponds to the background model, the black one to the data, and the grey area correspond to the selected green areas

In [ ]:
y = spectrum.data
bkgd = sf.residual(out.params,x)
plt.plot(x,y,"ko-",markersize=1.5,label="exp",markevery=10)
plt.fill_between(x,0,y,where=sum_boola,label="fit windows",color="0.8")
plt.xlabel("Energy (keV)",fontsize=22)
plt.xticks(fontsize=20)
# plt.ylim(0,1)
plt.yticks(fontsize=20)
plt.ylabel("Intensity",fontsize=22)
plt.plot(x,bkgd,"r-",linewidth=1,label="fit")
plt.legend(fontsize=22)

# VI. Select the interesting part of the spectrum

⚠️ A window will pop up with a red spectrum and a green area. With the green areas, you can select the energy range you want to quantify. Usually the low energy region (below 1keV) is not very well fitted by the background model. Hence you probably want to avoid the quantification of low energy characteristic X rays. 

In [ ]:
s_bkgd = hs.signals.Signal1D(bkgd)
s_bkgd.axes_manager[0].offset = offset
s_bkgd.axes_manager[0].scale = scale

spectrum.plot()
ax = spectrum._plot.signal_plot.ax
ax.add_line(matplotlib.lines.Line2D(spectrum._plot.axis.axis,bkgd.data))
roi = hs.roi.SpanROI(200*scale,400*scale)
roi.interactive(spectrum)

### Accepting the choice of quantification energy range

In [ ]:
charac_xrays = (spectrum.isig[roi.left:roi.right].data - s_bkgd.isig[roi.left:roi.right].data).clip(min=0)
new_offset = spectrum.isig[roi.left:roi.right].axes_manager[0].offset
new_size = spectrum.isig[roi.left:roi.right].axes_manager[0].size

# VIII. Fitting the characteristic Xrays

### Initialisation of the characteristic Xray model
⚠️ Put the list of elements (by atomic numbers) you want to quantify in ``elt_list``. 

``G_calib`` is a small offset you may want to add so that the peaks are at the right energy. 

Note that depending on the energy range you chose, not all the elements you put are fitted (e.g. Oxygen will not be taken into account if your energy range starts at 1 keV)

In [ ]:
elt_list = ["Sr", "Ti"]
G_calib = -0.005
db =  "default_xrays.json" # "300kv_xrays.json" 

params_dict = {
    "Abs" : {
        "thickness" : thickness,
        "density" : density,
        "toa" : take_off_angle
    },
    "Det" : detector
}

true_elt_list = []
for elt in elt_list : 
    mod_t = edxs.EDXS(new_offset,new_size,scale,{},db_name = db)
    mod_t.generate_g_matr(elements = [elt],norm=False, g_type = "no_brstlg")
    G_t = mod_t.G
    if G_t.shape[1] == 1 : 
        true_elt_list.append(elt)
    
mod = edxs.EDXS(new_offset + G_calib,new_size,scale,params_dict,db_name = db)
mod.generate_g_matr(elements = true_elt_list,norm=False, g_type = "no_brstlg")
G = mod.G
new_x = np.linspace(new_offset,new_size*scale+new_offset,num = new_size)
print("List of elements with Xray lines in the energy range :")
print(true_elt_list)

### Fitting the characteristic Xrays

In [ ]:
estimator = SmoothNMF(G=G,n_components= 1,max_iter=2000,force_simplex = True,tol = 1e-8,hspy_comp = False)
estimator.fit_transform(charac_xrays[:,np.newaxis], H=np.array([1])[:,np.newaxis])
W = estimator.W_

# IX. Plot the results

⚠️ The popping window will display the bkgd substracted experimental spectrum, the fitted characteristic Xrays model in black and red. Every element composing the model will be displayed with an offset : ``components_offset``. You can change its value below. 

If you want to display the names of the lines of an element that was used for quantification execute the next cells below before kill the window.

In [ ]:
components_offset = -0.1

In [ ]:
linestyles = [":","--","-."]

plt.plot(new_x,charac_xrays,"ko-",label="exp",markevery = 10)
plt.plot(new_x,G@W,"r",linewidth = 3,label="theo")
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.xlabel("energy (keV)",fontsize = 18)
plt.ylabel("Intensity", fontsize = 18)

for i in range(G.shape[1]) :
    ls_string = linestyles[i%len(linestyles)] + "C{}".format(i%9)
    plt.plot(new_x,G[:,i]*W[i]+components_offset,ls_string,label=str(true_elt_list[i]),linewidth=3)


plt.legend(fontsize=18)
plt.tight_layout()

# X. Printing the concentrations

Full summary of normalized concentrations.

In [ ]:
print_concentrations_from_W(W,elements = true_elt_list)

⚠️ You can remove some elements of the results, normalizing on the remaining elements.

In [ ]:
ignored_elts = ["Mg"]

inds = [true_elt_list.index(elt) for elt in ignored_elts]
remain_elt_list = [v for i,v in enumerate(true_elt_list) if i not in frozenset(inds)] 
remain_W = np.delete(W, inds, axis=0)
r = remain_W / remain_W.sum(axis=0)

print_concentrations_from_W(r,remain_elt_list)

# VI. Select the interesting part of the spectrum

⚠️ A window will pop up with a red spectrum and a green area. With the green areas, you can select the energy range you want to quantify. Usually the low energy region (below 1keV) is not very well fitted by the background model. Hence you probably want to avoid the quantification of low energy characteristic X rays. 

In [ ]:
s_bkgd = hs.signals.Signal1D(bkgd)
s_bkgd.axes_manager[0].offset = offset
s_bkgd.axes_manager[0].scale = scale

spectrum.plot()
ax = spectrum._plot.signal_plot.ax
ax.add_line(matplotlib.lines.Line2D(spectrum._plot.axis.axis,bkgd.data))
roi = hs.roi.SpanROI(200*scale,400*scale)
roi.interactive(spectrum)

In [ ]:
charac_xrays = (spectrum.isig[roi.left:roi.right].data - s_bkgd.isig[roi.left:roi.right].data).clip(min=0)
new_offset = spectrum.isig[roi.left:roi.right].axes_manager[0].offset
new_size = spectrum.isig[roi.left:roi.right].axes_manager[0].size

⚠️ If you want to fix the values of P you obtained in a previous fit put them in ``fixed_elts`` and execute the two following cells. 

For a two steps quantification : Chose a first energy range. Perform a first fit, which initialise P with all the elements. Then go back to the energy range selection, ignore the cell above, chose the fixed elements and perform the fit with fixed elements below. 

In [ ]:
print("Previous Values for W")
for i,elt in enumerate(true_elt_list) : 
    print(elt, " : ", W[i])

In [ ]:
fixed_elts = {"p1" : {"Cl" : 4.26588157e+18}}

dummy_spim = spim.EDS_espm(np.array([1]))
dummy_spim.add_elements(elements = true_elt_list)
dummy_spim.problem_type = "no_brstlg"

fixed_W = dummy_spim.set_fixed_W(fixed_elts)

In [ ]:
estimator = NMF(G=G,n_components= 1,max_iter=2000,force_simplex = True,tol = 1e-8,hspy_comp = False,fixed_W = fixed_W)
estimator.fit_transform(charac_xrays[:,np.newaxis], H=np.array([1])[:,np.newaxis])
new_W = estimator.W_

In [ ]:
print_concentrations_from_W(new_W,elements = true_elt_list)

In [ ]:
ignored_elts = ["Mg"]

inds = [true_elt_list.index(elt) for elt in ignored_elts]
remain_elt_list = [v for i,v in enumerate(true_elt_list) if i not in frozenset(inds)] 
remain_W = np.delete(new_W, inds, axis=0)
r = remain_W / remain_W.sum(axis=0)
print_concentrations_from_W(r,elements = true_elt_list)